In [1]:
import streamlit as st
from pathlib import Path
import pydeck as pdk
import folium
from streamlit_folium import st_folium
import numpy as np
import pandas as pd
from emission_explorer.Shapefile import subcountrymap
import geopandas as gpd
import requests
import io

In [2]:
url_to_download    = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_map_units.zip"
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})
r = requests.get(url_to_download, headers=headers)
shapefile = gpd.read_file(io.BytesIO(r.content))
shapes = shapefile['geometry']

In [3]:
shapefile.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 map unit,1,6,Fiji,FJI,0,2,Sovereign country,1,Fiji,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,Admin-0 map unit,1,3,United Republic of Tanzania,TZA,0,2,Sovereign country,1,United Republic of Tanzania,...,None,None,None,None,None,None,None,None,None,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Admin-0 map unit,1,7,Western Sahara,SAH,0,2,Indeterminate,1,Western Sahara,...,Unrecognized,Unrecognized,Unrecognized,None,None,Unrecognized,None,None,None,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Admin-0 map unit,1,2,Canada,CAN,0,2,Sovereign country,1,Canada,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,Admin-0 map unit,1,2,United States of America,US1,1,2,Country,1,United States of America,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [4]:
geo_layer = pdk.Layer(
    'GeoJsonLayer',
    data = shapefile.__geo_interface__,
    # get_position=['lng', 'lat'],
    pickable=True,
    auto_highlight=True,
    stroked=True,
    # # processes the data as a flat longitude-latitude pair
    get_polygon='-',
    get_fill_color=[250, 0, 0, 50],
    get_line_color=[0, 0, 0, 255],
    lineWidthMinPixels = 0.5
)

In [5]:
from ipywidgets import HTML

In [6]:
text = 'Test'

In [7]:
def get_countries(widget_instance, payload):
    global text
    text = payload

In [8]:
map = pdk.Deck(
    map_provider='carto',
    map_style='road',
    layers=[
       geo_layer 
    ],
    initial_view_state = pdk.ViewState(
        longitude=-1.415,
        latitude=52.2323,
        zoom=1
    )
)
map.deck_widget.on_click(get_countries)


In [2]:
import pydeck as pdk

DATA_URL = 'https://api.data.gov.sg/v1/transport/taxi-availability'
COLOR_RANGE = [
  [255, 255, 178, 25],
  [254, 217, 118, 85],
  [254, 178, 76, 127],
  [253, 141, 60, 170],
  [240, 59, 32, 212],
  [189, 0, 38, 255]
]

In [3]:
import pandas as pd
import requests

json = requests.get(DATA_URL).json()
df = pd.DataFrame(json["features"][0]["geometry"]["coordinates"])
df.columns = ['lng', 'lat']

viewport = pdk.data_utils.compute_view(df[['lng', 'lat']])
layer = pdk.Layer(
    'ScreenGridLayer',
    df,
    get_position=['lng', 'lat'],
    cell_size_pixels=20,
    color_range=COLOR_RANGE,
    pickable=True,
    auto_highlight=True)
r = pdk.Deck(layers=[layer], initial_view_state=viewport)

In [4]:
from ipywidgets import HTML

text = HTML(value='Points in viewport:')
layer = pdk.Layer('ScatterplotLayer', df, get_position=['lng', 'lat'], get_fill_color=[255, 0, 0], get_radius=100)
r = pdk.Deck(layer, initial_view_state=viewport)

In [6]:
def filter_by_bbox(row, west_lng, east_lng, north_lat, south_lat):
    return west_lng < row['lng'] < east_lng and south_lat < row['lat'] < north_lat


def filter_by_viewport(widget_instance, payload):
    try:
        west_lng, north_lat = payload['data']['nw']
        east_lng, south_lat = payload['data']['se']
        filtered_df = df[df.apply(lambda row: filter_by_bbox(row, west_lng, east_lng, north_lat, south_lat), axis=1)]
        text.value = 'Points in viewport: %s' % int(filtered_df.count()['lng'])
    except Exception as e:
        text.value = 'Error: %s' % e


r.deck_widget.on_view_state_change(filter_by_viewport)
display(text)
r.to_html()

HTML(value='Points in viewport:')